In [2]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from keras import optimizers
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image

import os
import numpy as np
import pandas as pd
import random
import glob
import math

from PIL import ImageFile
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
%matplotlib inline

# Loading the data

In [4]:
train_data_dir = 'data/train'

test_data_dir = 'data/test'
results_path = 'results/predictions'
results_name = 'predictions.csv'

In [5]:
# dimensions of our images.
img_width, img_height = 150, 150

# Found 24198 images belonging to 2 classes.
# Found 802 images belonging to 2 classes.
nb_train_samples = 1481

epochs = 20
batch_size = 16

In [6]:
base_model = applications.VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_width, img_height, 3))
print('Model Loaded')

Model Loaded


In [7]:
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='sigmoid'))

In [8]:
# add the model on top of the convolutional base
model = Model(input=base_model.input, output=top_model(base_model.output))

C:\Users\Flo\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`
  


In [9]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

In [10]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [11]:
# checkpoint
filepath="vgg-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [12]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1. / 255)
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [15]:
itr = train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=1481,
    class_mode='categorical')

Found 1481 images belonging to 3 classes.


In [17]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [18]:
batchx, batchy = itr.next()

In [19]:
print(batchx.shape)
print(batchy.shape)

(1481, 150, 150, 3)
(1481, 3)


In [20]:
print(batchy)

[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 ..., 
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]]


In [23]:
# zero center
batchx -= np.mean(batchx, axis = 0)

In [28]:
history = model.fit(batchx, batchy, batch_size=16, epochs=20, callbacks=callbacks_list, validation_split=0.22, shuffle=True, initial_epoch=0)

Train on 1155 samples, validate on 326 samples
Epoch 1/20
1155/1155 [==============================] - 19s - loss: 1.2559 - acc: 0.2494 - val_loss: 1.2037 - val_acc: 0.1656
Epoch 2/20
1155/1155 [==============================] - 14s - loss: 1.2464 - acc: 0.2494 - val_loss: 1.2037 - val_acc: 0.1656
Epoch 3/20
1155/1155 [==============================] - 14s - loss: 1.2754 - acc: 0.2329 - val_loss: 1.2037 - val_acc: 0.1656
Epoch 4/20
1155/1155 [==============================] - 27s - loss: 1.2627 - acc: 0.2502 - val_loss: 1.2037 - val_acc: 0.1656
Epoch 5/20
1155/1155 [==============================] - 37s - loss: 1.2550 - acc: 0.2511 - val_loss: 1.2037 - val_acc: 0.1656
Epoch 6/20
1155/1155 [==============================] - 14s - loss: 1.2582 - acc: 0.2485 - val_loss: 1.2037 - val_acc: 0.1656
Epoch 7/20
1155/1155 [==============================] - 14s - loss: 1.2476 - acc: 0.2667 - val_loss: 1.2037 - val_acc: 0.1656
Epoch 8/20
1155/1155 [==============================] - 14s - loss: 1.2

KeyboardInterrupt: 

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_vgg.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_vgg.h5")
print("Saved model to disk")

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()